In [ ]:
import pandas as pd
import math as math
from collections import defaultdict
import scipy 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
client_file = 'dataset/client.csv'
district_file = 'dataset/district.csv'
account_file = 'dataset/account.csv'
disp_file = 'dataset/disp.csv'
card_dev_file = 'dataset/card_dev.csv'
loan_dev_file = 'dataset/loan_dev.csv'
trans_dev_file = 'dataset/trans_dev.csv'

In [ ]:
client_df = pd.read_csv(client_file, sep=";")
district_df = pd.read_csv(district_file, sep=";")
account_df = pd.read_csv(account_file, sep=";")
disp_df = pd.read_csv(disp_file, sep=";")
card_dev_df = pd.read_csv(card_dev_file, sep=";")
loan_dev_df = pd.read_csv(loan_dev_file, sep=";")
trans_dev_df = pd.read_csv(trans_dev_file, sep=";", dtype={'bank': str, 'account': str})


In [ ]:
#months to 0:x indexes

month_index = defaultdict(int)
index = 0
for y in range(93, 98):
    for m in range(1,13):
        if(m < 10):
            month_index[str(y) + '0' + str(m)] = index
        else:
            month_index[str(y) + str(m)] = index
        index += 1

print(month_index)



In [ ]:
print([trans_dev_df["account_id"][i] for i in trans_dev_df.index[:10]])

In [ ]:
#cria lista das datas de um loan fraudolento
x_default_loan = []
for i in loan_dev_df.index:
    date = str(loan_dev_df['date'][i])[0:4]
    status = str(loan_dev_df['status'][i])
    if(status == '-1'): x_default_loan.append(month_index[date])

print(x_default_loan)

In [ ]:
#cria mapa key: month value: number accounts
month_cnt = defaultdict(int)
for i in account_df.index:
    date = str(account_df['date'][i])[0:4]
    month_cnt[date] += 1

print(month_cnt) 
x = []
y = []
for k,v in month_cnt.items():
    x.append(month_index[k])
    y.append(v)


plt.plot(x,y)
plt.show()

In [ ]:
#cria mapa key: month value: transactions done
month_cnt = defaultdict(int)
for i in trans_dev_df.index:
    date = str(trans_dev_df['date'][i])[0:4]
    month_cnt[date] += 1

print(month_cnt) 
x = []
y = []
for k,v in month_cnt.items():
    x.append(month_index[k])
    y.append(v)

#red lines
for xc in x_default_loan:
    plt.axvline(x=xc,ymax = 140, color='red')

plt.plot(x,y)
plt.show()

In [ ]:
# cria mapa do cash flow de cada mes para cada conta

mp = defaultdict(lambda: defaultdict(int))
for i in trans_dev_df.index:
    acount = trans_dev_df['account_id'][i]
    date = str(trans_dev_df['date'][i])[0:4]
    amount = 0
    if(trans_dev_df['type'][i] == "credit"): amount = trans_dev_df["amount"][i]
    elif(trans_dev_df['type'][i] == "withdrawal" ): amount = - trans_dev_df["amount"][i]
    mp[acount][date] += amount

print(mp)


In [ ]:
#histogram number of months of acounts
arr = []
for k,v in mp.items():
    arr.append(len(v))

hist, bin_edges = np.histogram(arr, bins = range(50))

print ("No. of points in each bin : ", hist) 
print ("Size of the bins          : ", bin_edges)

plt.bar(bin_edges[:-1], hist, width = 1) 
plt.xlim(min(bin_edges), max(bin_edges)) 
plt.show() 

#print(arr)

In [ ]:
#cria mapa com media de cash flow ate um certo mes para cada conta

#key1: acountId key2: date in format yymm all str
cash_flow_mean = defaultdict(lambda: defaultdict(int))
#mean_until_month = defaultdict()
for k1,v1 in mp.items():
    #print("key: " + str(k1) + " V: " + str(v1))
    total = 0
    cnt = 0
    for k2,v2 in v1.items():
        #print("key1: " + str(k1) + " Key2: " + str(k2))
        total += int(v2)
        cnt += 1
        cash_flow_mean[str(k1)][str(k2)] = total / cnt


In [ ]:
#scatter plot cash_flow loan amount
x = [] #cash_flow
y = [] #loan payment
c = [] #color

for i in loan_dev_df.index:
    acount = str(loan_dev_df['account_id'][i])

    date = str(loan_dev_df['date'][i])[0:4]
    if(date[2:4] == '01'):
        dt = int(date[0:2])
        dt -= 1
        date = str(dt) + '12'
    else:
        dt = int(date[2:4])
        dt -= 1
        if(dt < 10):
            date = date[0:2] + '0' + str(dt)
        else:
            date = date[0:2] + str(dt)

    cash_flow_mean_val = int(cash_flow_mean[acount][date]) 
    x.append(cash_flow_mean_val)
    #if(cash_flow_mean_val == 0):
        #print("Acount: " + acount + " " + date)
        #print(cash_flow_mean[acount])
    
    amount = int(loan_dev_df['amount'][i])
    y.append(amount)

    status = str(loan_dev_df['status'][i])
    if(status == '-1'): c.append("Red") 
    else: c.append("Green")

plt.scatter(x, y, c=c)
plt.show()


In [ ]:
#scatter plot cash_flow loan payment
x = [] #cash_flow
y = [] #loan payment
c = [] #color

for i in loan_dev_df.index:
    acount = str(loan_dev_df['account_id'][i])

    date = str(loan_dev_df['date'][i])[0:4]
    if(date[2:4] == '01'):
        dt = int(date[0:2])
        dt -= 1
        date = str(dt) + '12'
    else:
        dt = int(date[2:4])
        dt -= 1
        if(dt < 10):
            date = date[0:2] + '0' + str(dt)
        else:
            date = date[0:2] + str(dt)

    cash_flow_mean_val = int(cash_flow_mean[acount][date]) 
    x.append(cash_flow_mean_val)
    #if(cash_flow_mean_val == 0):
        #print("Acount: " + acount + " " + date)
        #print(cash_flow_mean[acount])
    
    amount = int(loan_dev_df['payments'][i])
    y.append(amount)

    status = str(loan_dev_df['status'][i])
    if(status == '-1'): c.append("Red") 
    else: c.append("Green")

plt.scatter(x, y, c=c)
plt.show()
